In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Overview

This tutorial demonstrates how to use the Vertex AI SDK for Python to set up Vertex AI Model Monitoring V2 for your model.

### Objective

In this tutorial, you'll complete the following steps:

- Upload a custom model to Vertex AI Model Registry.
- Create a model monitor.
- Create Vertex AI batch prediction job.
- Run an on-demand model monitoring job to analyze data drift between the batch prediction job results and the training dataset.
- Create another Vertex AI batch prediction job.
- Run an on-demand model monitoring job to analyze data drift between the batch prediction job results and the previous batch prediction job.
- Run an on-demand model monitoring job to analyze the feature attribution drift between the batch prediction job results and a baseline dataset in Google Cloud Storage.


### Costs

Vertex AI Model Monitoring v2 is free during the public preview period, but you will still be billed for the following Google Cloud services:

* [BigQuery](https://cloud.google.com/bigquery/pricing)
* [Cloud Storage](https://cloud.google.com/storage/pricing)
* [Vertex AI Online Prediction](https://cloud.google.com/vertex-ai/pricing#prediction-prices)
* [Vertex AI Batch Explanation Job](https://cloud.google.com/vertex-ai/pricing#prediction-prices) (if you run the feature attribution drift example).

## Getting Started

### Install Vertex AI SDK and other required packages

In [1]:
!pip install \
    google-cloud-bigquery \
    pandas==1.5.3 \
    pandas-gbq \
    pyarrow \
    tensorflow-data-validation \
    google-cloud-aiplatform \
    tensorflow-metadata \
    google-api-core \
    google-auth \
    google-cloud-core \
    google-resumable-media \
    tfx-bsl

  Using cached tensorflow_data_validation-1.16.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
INFO: pip is looking at multiple versions of tensorflow-data-validation to determine which version is compatible with other requirements. This could take a while.
  Using cached tensorflow_data_validation-1.16.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached tensorflow_data_validation-1.15.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached tensorflow_data_validation-1.15.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached db_dtypes-1.4.3-py3-none-any.whl.metadata (3.0 kB)
INFO: pip is still looking at multiple versions of tensorflow-data-validation to determine which version is compatible with other requirements. This could take a while.
  Using cached db_dtypes-1.4.2-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached pyfarmhash-0.3.2-

Check that the version of google-cloud-aiplatform is 1.51.0 or later.

In [2]:
from google.cloud import aiplatform

aiplatform.__version__

'1.120.0'

### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [3]:
import sys

if not "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.

In [1]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [3]:
# Make sure to replace the value of `PROJECT_ID` with your GCP project ID and `LOCATION` with the region associated with your lab.
PROJECT_ID = "qwiklabs-gcp-01-89e8598a3363"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}


import os

import vertexai

! gcloud config set project $PROJECT_ID
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
! gcloud config set ai/region $LOCATION

vertexai.init(project=PROJECT_ID, location=LOCATION)

Updated property [core/project].
Updated property [ai/region].


## Start Model Monitoring tutorial

### Step 1: Create a Cloud Storage bucket

Create a Cloud Storage bucket to store intermediate artifacts such as datasets.

In [4]:
# Create a Cloud Storage bucket
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [5]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://your-bucket-name-qwiklabs-gcp-01-89e8598a3363-unique/...


### Step 2: Prepare a model in Vertex AI Model Registry

You can register a model in Vertex AI Model Registry with its artifacts, enabling you to perform online serving or batch prediction. Alternatively, you can register a referenced/placeholder model that includes only the model's name.
In this notebook, you register a model with artifacts because you'll run a batch prediction job.

In [6]:
import google.cloud.aiplatform as aiplatform

MODEL_PATH = "gs://cloud-samples-data/vertex-ai/model-deployment/models/churn"
MODEL_NAME = "churn"
IMAGE = "us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-5:latest"

model = aiplatform.Model.upload(
    display_name=MODEL_NAME,
    artifact_uri=MODEL_PATH,
    serving_container_image_uri=IMAGE,
    sync=True,
)

MODEL_ID = model.resource_name.split("/")[-1]

### Step 3: Create a model monitor

Create a model monitor to associate monitoring details with a model version that has been register in Vertex AI Model Registry.

#### Define Model Monitoring Schema

The monitoring schema is required for model monitors. It includes the names of input features, prediction outputs and, if available, ground truths, along with their respective data type.

**Note: For AutoML tables (regression and classification), defining the schema is optional. The schema is automatically fetched when available. If Vertex AI cannot get the schema information, you must provide it.**

In [7]:
from vertexai.resources.preview import ml_monitoring

MODEL_MONITORING_SCHEMA = ml_monitoring.spec.ModelMonitoringSchema(
    feature_fields=[
        ml_monitoring.spec.FieldSchema(name="user_pseudo_id", data_type="string"),
        ml_monitoring.spec.FieldSchema(name="country", data_type="string"),
        ml_monitoring.spec.FieldSchema(name="operating_system", data_type="string"),
        ml_monitoring.spec.FieldSchema(name="cnt_user_engagement", data_type="integer"),
        ml_monitoring.spec.FieldSchema(
            name="cnt_level_start_quickplay", data_type="integer"
        ),
        ml_monitoring.spec.FieldSchema(
            name="cnt_level_end_quickplay", data_type="integer"
        ),
        ml_monitoring.spec.FieldSchema(
            name="cnt_level_complete_quickplay", data_type="integer"
        ),
        ml_monitoring.spec.FieldSchema(
            name="cnt_level_reset_quickplay", data_type="integer"
        ),
        ml_monitoring.spec.FieldSchema(name="cnt_post_score", data_type="integer"),
        ml_monitoring.spec.FieldSchema(
            name="cnt_spend_virtual_currency", data_type="integer"
        ),
        ml_monitoring.spec.FieldSchema(name="cnt_ad_reward", data_type="integer"),
        ml_monitoring.spec.FieldSchema(
            name="cnt_challenge_a_friend", data_type="integer"
        ),
        ml_monitoring.spec.FieldSchema(
            name="cnt_completed_5_levels", data_type="integer"
        ),
        ml_monitoring.spec.FieldSchema(name="cnt_use_extra_steps", data_type="integer"),
        ml_monitoring.spec.FieldSchema(name="month", data_type="categorical"),
        ml_monitoring.spec.FieldSchema(name="julianday", data_type="integer"),
        ml_monitoring.spec.FieldSchema(name="dayofweek", data_type="integer"),
    ],
    ground_truth_fields=[
        ml_monitoring.spec.FieldSchema(name="churned", data_type="categorical")
    ],
    prediction_fields=[
        ml_monitoring.spec.FieldSchema(
            name="predicted_churned", data_type="categorical"
        )
    ],
)

2025-10-15 14:19:57.426995: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-15 14:19:57.432662: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-15 14:19:57.451885: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-15 14:19:57.487349: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-15 14:19:57.487411: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-15 14:19:57.515133: I tensorflow/core/platform/cpu_feature_guard.cc:

#### Define training dataset

The training dataset can serve as the baseline dataset to calculate monitoring metrics. You can register the training dataset in the model monitor.

In [8]:
from vertexai.resources.preview import ml_monitoring

# Copy files to your projects gs bucket to avoid permission issues.
# Ignore any error(s) for bucket already exists.
PUBLIC_TRAINING_DATASET = (
    "gs://cloud-samples-data/vertex-ai/model-monitoring/churn/churn_training.csv"
)
TRAINING_URI = f"{BUCKET_URI}/model-monitoring/churn/churn_training.csv"

! gsutil copy $PUBLIC_TRAINING_DATASET $TRAINING_URI

TRAINING_DATASET = ml_monitoring.spec.MonitoringInput(
    gcs_uri=TRAINING_URI, data_format="csv"
)

Copying gs://cloud-samples-data/vertex-ai/model-monitoring/churn/churn_training.csv [Content-Type=text/csv]...
/ [1 files][717.9 KiB/717.9 KiB]                                                
Operation completed over 1 objects/717.9 KiB.                                    


#### Create a model monitor resource

A model monitor is a top-level resource to manage your metrics and model monitoring jobs.

In [9]:
from vertexai.resources.preview import ml_monitoring

my_model_monitor = ml_monitoring.ModelMonitor.create(
    project=PROJECT_ID,
    location=LOCATION,
    display_name="churn_model_monitor",
    model_name=model.resource_name,
    model_version_id="1",
    training_dataset=TRAINING_DATASET,
    model_monitoring_schema=MODEL_MONITORING_SCHEMA,
)
MODEL_MONITOR_ID = my_model_monitor.name
print(f"MODEL MONITOR {MODEL_MONITOR_ID} created.")

INFO:vertexai.resources.preview.ml_monitoring.model_monitors:Creating ModelMonitor
INFO:vertexai.resources.preview.ml_monitoring.model_monitors:Create ModelMonitor backing LRO: projects/684754974539/locations/us-central1/modelMonitors/2411541261015056384/operations/4985967833880985600


MODEL MONITOR 2411541261015056384 created.


INFO:vertexai.resources.preview.ml_monitoring.model_monitors:ModelMonitor created. Resource name: projects/684754974539/locations/us-central1/modelMonitors/2411541261015056384
INFO:vertexai.resources.preview.ml_monitoring.model_monitors:To use this ModelMonitor in another session:
INFO:vertexai.resources.preview.ml_monitoring.model_monitors:model_monitor = aiplatform.ModelMonitor('projects/684754974539/locations/us-central1/modelMonitors/2411541261015056384')
INFO:vertexai.resources.preview.ml_monitoring.model_monitors:https://console.cloud.google.com/vertex-ai/model-monitoring/locations/us-central1/model-monitors/2411541261015056384?project=qwiklabs-gcp-01-89e8598a3363


### Step 4: Run an on-demand model monitoring job

#### Define the monitoring objective configs

For tabular models, Model Monitoring supports the following objectives:

*   **Input feature drift detection**

    Model Monitoring offers drift analysis for both categorical and numeric feature types, with the following supported metrics:

    *    Categorical Feature: `Jensen Shannon Divergence`, `L Infinity`
    *    Numeric Feature: `Jensen Shannon Divergence`

    You can choose to analyze only the features of interest by specifying them in the `features` fields of the `ml_monitoring.spec.DataDriftSpec` specification. If not specified, all input features in the model schema are analyzed. Additionally, you have the option to set default thresholds for categorical or numeric features, or you can specify thresholds for individual features. If the detected drift surpasses a threshold, an alert is sent through email or another notification channel.

*  **Prediction output drift detection**

    Similar to input feature drift detection, prediction output drift detection identifies data drift in the prediction outputs.

*   **Feature attribution drift detection**

    Model Monitoring leverages Vertex Explainable AI to monitor feature attributions. Explainable AI enables you to understand the relative contribution of each feature to a resulting prediction. In essence, it assesses the magnitude of each feature's influence.
    You must configure the `Explanation` specification with the feature attribution objectives configuration.


Input feature drift specification

In [10]:
from vertexai.resources.preview import ml_monitoring

FEATURE_THRESHOLDS = {
    "country": 0.003,
    "cnt_user_engagement": 0.004,
}

FEATURE_DRIFT_SPEC = ml_monitoring.spec.DataDriftSpec(
    categorical_metric_type="l_infinity",
    numeric_metric_type="jensen_shannon_divergence",
    default_categorical_alert_threshold=0.2,
    default_numeric_alert_threshold=0.3,
    feature_alert_thresholds=FEATURE_THRESHOLDS,
)

Prediction output drift specification

In [11]:
PREDICTION_OUTPUT_DRIFT_SPEC = ml_monitoring.spec.DataDriftSpec(
    categorical_metric_type="l_infinity",
    numeric_metric_type="jensen_shannon_divergence",
    default_categorical_alert_threshold=0.1,
    default_numeric_alert_threshold=0.1,
)

Feature attribution specification

In [12]:
FEATURE_ATTRIBUTION_SPEC = ml_monitoring.spec.FeatureAttributionSpec(
    default_alert_threshold=0.0003,
    feature_alert_thresholds={"cnt_ad_reward": 0.0001},
)

#### Define the alert notification and metrics output spec.

Model Monitoring supports the following notification methods:

*   Email
*   [Notification Channel](https://cloud.google.com/monitoring/support/notification-options)
*   [Cloud Logging](https://cloud.google.com/logging/docs)  

This notebook uses email as an example.

Export generated metrics to the Google Cloud Storage location that you specified or, if you don't specify a location, Vertex AI creates a default bucket to use.

In [13]:
import os

from vertexai.resources.preview import ml_monitoring

EMAIL = "stripling@google.com"  # @param {type:"string"}
if os.getenv("IS_TESTING"):
    EMAIL = "noreply@google.com"

NOTIFICATION_SPEC = ml_monitoring.spec.NotificationSpec(
    user_emails=[EMAIL],
)

OUTPUT_SPEC = ml_monitoring.spec.OutputSpec(gcs_base_dir=BUCKET_URI)

#### Run Model Monitoring Jobs

##### **Example 1: Detect feature drift by comparing a batch prediction job with the training dataset.**

Let's first create a batch prediction job.

In [14]:
BP_INPUT_URI_1 = f"{BUCKET_URI}/model-monitoring/churn/churn_bp_input_1.jsonl"
! gsutil copy gs://cloud-samples-data/vertex-ai/model-monitoring/churn/churn_bp_input_1.jsonl $BP_INPUT_URI_1

Copying gs://cloud-samples-data/vertex-ai/model-monitoring/churn/churn_bp_input_1.jsonl [Content-Type=application/octet-stream]...
/ [1 files][223.9 KiB/223.9 KiB]                                                
Operation completed over 1 objects/223.9 KiB.                                    


In [15]:
batch_prediction_job_1 = model.batch_predict(
    generate_explanation=True,
    job_display_name="bp_example_1",
    instances_format="jsonl",
    machine_type="e2-standard-4",
    gcs_source=[BP_INPUT_URI_1],
    gcs_destination_prefix=f"{BUCKET_URI}/bp_output",
    sync=True,
)

In [16]:
import pandas as pd
from vertexai.resources.preview import ml_monitoring

TIMESTAMP = pd.Timestamp.utcnow().strftime("%Y%m%d%H%M%S")
JOB_DISPLAY_NAME = f"churn_model_monitoring_job_{TIMESTAMP}"
TARGET_DATASET = ml_monitoring.spec.MonitoringInput(
    batch_prediction_job=batch_prediction_job_1.resource_name
)
model_monitoring_job_1 = my_model_monitor.run(
    display_name=JOB_DISPLAY_NAME,
    baseline_dataset=TRAINING_DATASET,
    target_dataset=TARGET_DATASET,
    tabular_objective_spec=ml_monitoring.spec.TabularObjective(
        # Input feature drift spec.
        feature_drift_spec=FEATURE_DRIFT_SPEC
    ),
    notification_spec=NOTIFICATION_SPEC,
    output_spec=OUTPUT_SPEC,
)

INFO:vertexai.resources.preview.ml_monitoring.model_monitors:Creating ModelMonitoringJob
INFO:vertexai.resources.preview.ml_monitoring.model_monitors:ModelMonitoringJob created. Resource name: projects/684754974539/locations/us-central1/modelMonitors/2411541261015056384/modelMonitoringJobs/8595201252697047040
INFO:vertexai.resources.preview.ml_monitoring.model_monitors:To use this ModelMonitoringJob in another session:
INFO:vertexai.resources.preview.ml_monitoring.model_monitors:model_monitoring_job = aiplatform.ModelMonitoringJob('projects/684754974539/locations/us-central1/modelMonitors/2411541261015056384/modelMonitoringJobs/8595201252697047040')
INFO:vertexai.resources.preview.ml_monitoring.model_monitors:ModelMonitoringJob projects/684754974539/locations/us-central1/modelMonitors/2411541261015056384/modelMonitoringJobs/8595201252697047040 current state:
JobState.JOB_STATE_RUNNING


##### **Example 2: Detect feature drift and prediction output drift by comparing a batch prediction job with a previous batch prediction job result.**

You can set up multiple objectives within a single model monitoring job. All metrics are computed by using the same baseline and target dataset.

Create another batch prediction job and compare it with the batch prediction job created previously.

In [17]:
BP_INPUT_URI_2 = f"{BUCKET_URI}/model-monitoring/churn/churn_bp_input_2.jsonl"
! gsutil copy gs://cloud-samples-data/vertex-ai/model-monitoring/churn/churn_bp_input_2.jsonl $BP_INPUT_URI_2

INFO:vertexai.resources.preview.ml_monitoring.model_monitors:ModelMonitoringJob projects/684754974539/locations/us-central1/modelMonitors/2411541261015056384/modelMonitoringJobs/8595201252697047040 current state:
JobState.JOB_STATE_RUNNING


Copying gs://cloud-samples-data/vertex-ai/model-monitoring/churn/churn_bp_input_2.jsonl [Content-Type=application/octet-stream]...
/ [1 files][443.4 KiB/443.4 KiB]                                                
Operation completed over 1 objects/443.4 KiB.                                    


In [18]:
batch_prediction_job_2 = model.batch_predict(
    job_display_name="bp_example_2",
    instances_format="jsonl",
    machine_type="e2-standard-4",
    gcs_source=[BP_INPUT_URI_2],
    gcs_destination_prefix=f"{BUCKET_URI}/bp_output",
    sync=True,
)

INFO:vertexai.resources.preview.ml_monitoring.model_monitors:ModelMonitoringJob projects/684754974539/locations/us-central1/modelMonitors/2411541261015056384/modelMonitoringJobs/8595201252697047040 current state:
JobState.JOB_STATE_RUNNING
INFO:vertexai.resources.preview.ml_monitoring.model_monitors:ModelMonitoringJob projects/684754974539/locations/us-central1/modelMonitors/2411541261015056384/modelMonitoringJobs/8595201252697047040 current state:
JobState.JOB_STATE_RUNNING
INFO:vertexai.resources.preview.ml_monitoring.model_monitors:ModelMonitoringJob projects/684754974539/locations/us-central1/modelMonitors/2411541261015056384/modelMonitoringJobs/8595201252697047040 current state:
JobState.JOB_STATE_RUNNING
INFO:vertexai.resources.preview.ml_monitoring.model_monitors:ModelMonitoringJob run completed. Resource name: projects/684754974539/locations/us-central1/modelMonitors/2411541261015056384/modelMonitoringJobs/8595201252697047040


In [19]:
import pandas as pd
from vertexai.resources.preview import ml_monitoring

TIMESTAMP = pd.Timestamp.utcnow().strftime("%Y%m%d%H%M%S")
JOB_DISPLAY_NAME = f"churn_model_monitoring_job_{TIMESTAMP}"
BASELINE_DATASET = ml_monitoring.spec.MonitoringInput(
    batch_prediction_job=batch_prediction_job_1.resource_name
)
TARGET_DATASET = ml_monitoring.spec.MonitoringInput(
    batch_prediction_job=batch_prediction_job_2.resource_name
)
model_monitoring_job_2 = my_model_monitor.run(
    display_name=JOB_DISPLAY_NAME,
    baseline_dataset=BASELINE_DATASET,
    target_dataset=TARGET_DATASET,
    tabular_objective_spec=ml_monitoring.spec.TabularObjective(
        # Input feature drift spec.
        feature_drift_spec=FEATURE_DRIFT_SPEC,
        # Prediction output drift spec.
        prediction_output_drift_spec=PREDICTION_OUTPUT_DRIFT_SPEC,
    ),
    notification_spec=NOTIFICATION_SPEC,
    output_spec=OUTPUT_SPEC,
)

INFO:vertexai.resources.preview.ml_monitoring.model_monitors:Creating ModelMonitoringJob
INFO:vertexai.resources.preview.ml_monitoring.model_monitors:ModelMonitoringJob created. Resource name: projects/684754974539/locations/us-central1/modelMonitors/2411541261015056384/modelMonitoringJobs/8743820040400273408
INFO:vertexai.resources.preview.ml_monitoring.model_monitors:To use this ModelMonitoringJob in another session:
INFO:vertexai.resources.preview.ml_monitoring.model_monitors:model_monitoring_job = aiplatform.ModelMonitoringJob('projects/684754974539/locations/us-central1/modelMonitors/2411541261015056384/modelMonitoringJobs/8743820040400273408')
INFO:vertexai.resources.preview.ml_monitoring.model_monitors:ModelMonitoringJob projects/684754974539/locations/us-central1/modelMonitors/2411541261015056384/modelMonitoringJobs/8743820040400273408 current state:
JobState.JOB_STATE_RUNNING


##### **Example 3: Feature attribution drift detection, compares the batch prediction job with a GCS baseline dataset**

For feature attribution monitoring, the dataset is sent to the Vertex AI batch explanation job in the following way:

*   Google Cloud Storage -> Sent directly as input to Vertex AI batch explanation job.
*   BigQuery table -> Sent directly as input to Vertex AI batch explanation job.
*   BigQuery Query -> Not supported.
*   Vertex AI batch explanation job -> Input of batch prediction job is used as input for the Vertex AI batch explanation job.
*   Vertex AI endpoint logging -> Request logging is used as input for Vertex AI batch explanation job.

Check that these datasets meet the requirements for a Vertex AI batch explanation job.

###### Generate model metadata for Vertex Explainable AI
You must specify the explanation specification to use a Vertex AI batch explanation job. Run the following cell to extract metadata from the exported model, which is needed for generating the explanations for a prediction request.

In [20]:
from google.cloud.aiplatform_v1beta1.types import (ExplanationMetadata,
                                                   ExplanationParameters,
                                                   ExplanationSpec)

EXPLANATION_SPEC = ExplanationSpec(
    parameters=ExplanationParameters(
        {"sampled_shapley_attribution": {"path_count": 2}}
    ),
    metadata=ExplanationMetadata(
        inputs={
            "cnt_ad_reward": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "cnt_ad_reward",
                    "encoding": "IDENTITY",
                    "modality": "numeric",
                }
            ),
            "cnt_challenge_a_friend": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "cnt_challenge_a_friend",
                    "encoding": "IDENTITY",
                    "modality": "numeric",
                }
            ),
            "cnt_completed_5_levels": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "cnt_completed_5_levels",
                    "encoding": "IDENTITY",
                    "modality": "numeric",
                }
            ),
            "cnt_level_complete_quickplay": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "cnt_level_complete_quickplay",
                    "encoding": "IDENTITY",
                    "modality": "numeric",
                }
            ),
            "cnt_level_end_quickplay": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "cnt_level_end_quickplay",
                    "encoding": "IDENTITY",
                    "modality": "numeric",
                }
            ),
            "cnt_level_reset_quickplay": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "cnt_level_reset_quickplay",
                    "encoding": "IDENTITY",
                    "modality": "numeric",
                }
            ),
            "cnt_level_start_quickplay": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "cnt_level_start_quickplay",
                    "encoding": "IDENTITY",
                    "modality": "numeric",
                }
            ),
            "cnt_post_score": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "cnt_post_score",
                    "encoding": "IDENTITY",
                    "modality": "numeric",
                }
            ),
            "cnt_spend_virtual_currency": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "cnt_spend_virtual_currency",
                    "encoding": "IDENTITY",
                    "modality": "numeric",
                }
            ),
            "cnt_use_extra_steps": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "cnt_use_extra_steps",
                    "encoding": "IDENTITY",
                    "modality": "numeric",
                }
            ),
            "cnt_user_engagement": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "cnt_user_engagement",
                    "encoding": "IDENTITY",
                    "modality": "numeric",
                }
            ),
            "country": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "country",
                    "encoding": "IDENTITY",
                    "modality": "categorical",
                }
            ),
            "dayofweek": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "dayofweek",
                    "encoding": "IDENTITY",
                    "modality": "numeric",
                }
            ),
            "julianday": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "julianday",
                    "encoding": "IDENTITY",
                    "modality": "numeric",
                }
            ),
            "language": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "language",
                    "encoding": "IDENTITY",
                    "modality": "categorical",
                }
            ),
            "month": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "month",
                    "encoding": "IDENTITY",
                    "modality": "numeric",
                }
            ),
            "operating_system": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "operating_system",
                    "encoding": "IDENTITY",
                    "modality": "categorical",
                }
            ),
            "user_pseudo_id": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "user_pseudo_id",
                    "encoding": "IDENTITY",
                    "modality": "categorical",
                }
            ),
        },
        outputs={
            "churned_probs": ExplanationMetadata.OutputMetadata(
                {"output_tensor_name": "churned_probs"}
            )
        },
    ),
)

INFO:vertexai.resources.preview.ml_monitoring.model_monitors:ModelMonitoringJob projects/684754974539/locations/us-central1/modelMonitors/2411541261015056384/modelMonitoringJobs/8743820040400273408 current state:
JobState.JOB_STATE_RUNNING


In [21]:
FEATURE_ATTRIBUTION_BASELINE_DATASET = (
    f"{BUCKET_URI}/model-monitoring/churn/churn_no_ground_truth.jsonl"
)
! gsutil cp gs://cloud-samples-data/vertex-ai/model-monitoring/churn/churn_no_ground_truth.jsonl $FEATURE_ATTRIBUTION_BASELINE_DATASET

Copying gs://cloud-samples-data/vertex-ai/model-monitoring/churn/churn_no_ground_truth.jsonl [Content-Type=application/octet-stream]...
/ [1 files][  3.5 MiB/  3.5 MiB]                                                
Operation completed over 1 objects/3.5 MiB.                                      


In [22]:
import pandas as pd
from vertexai.resources.preview import ml_monitoring

TIMESTAMP = pd.Timestamp.utcnow().strftime("%Y%m%d%H%M%S")
JOB_DISPLAY_NAME = f"churn_model_monitoring_job_{TIMESTAMP}"
BASELINE_DATASET = ml_monitoring.spec.MonitoringInput(
    gcs_uri=FEATURE_ATTRIBUTION_BASELINE_DATASET, data_format="jsonl"
)
TARGET_DATASET = ml_monitoring.spec.MonitoringInput(
    batch_prediction_job=batch_prediction_job_2.resource_name
)
model_monitoring_job_3 = my_model_monitor.run(
    display_name=JOB_DISPLAY_NAME,
    baseline_dataset=BASELINE_DATASET,
    target_dataset=TARGET_DATASET,
    tabular_objective_spec=ml_monitoring.spec.TabularObjective(
        # Feature attribution spec.
        feature_attribution_spec=FEATURE_ATTRIBUTION_SPEC
    ),
    # You must have a Explanation spec for feature attribution monitoring.
    # You can specify the explanation spec in the Model, Model monitor, or the Model monitoring job.
    explanation_spec=EXPLANATION_SPEC,
    notification_spec=NOTIFICATION_SPEC,
    output_spec=OUTPUT_SPEC,
)

INFO:vertexai.resources.preview.ml_monitoring.model_monitors:Creating ModelMonitoringJob
INFO:vertexai.resources.preview.ml_monitoring.model_monitors:ModelMonitoringJob created. Resource name: projects/684754974539/locations/us-central1/modelMonitors/2411541261015056384/modelMonitoringJobs/3436327879544143872
INFO:vertexai.resources.preview.ml_monitoring.model_monitors:To use this ModelMonitoringJob in another session:
INFO:vertexai.resources.preview.ml_monitoring.model_monitors:model_monitoring_job = aiplatform.ModelMonitoringJob('projects/684754974539/locations/us-central1/modelMonitors/2411541261015056384/modelMonitoringJobs/3436327879544143872')
INFO:vertexai.resources.preview.ml_monitoring.model_monitors:ModelMonitoringJob projects/684754974539/locations/us-central1/modelMonitors/2411541261015056384/modelMonitoringJobs/8743820040400273408 current state:
JobState.JOB_STATE_RUNNING
INFO:vertexai.resources.preview.ml_monitoring.model_monitors:ModelMonitoringJob projects/684754974539/

##### List Model Monitoring Jobs

In [23]:
my_model_monitor.list_jobs()

[name: "projects/684754974539/locations/us-central1/modelMonitors/2411541261015056384/modelMonitoringJobs/3436327879544143872"
display_name: "churn_model_monitoring_job_20251015150340"
model_monitoring_spec {
  objective_spec {
    tabular_objective {
      feature_attribution_spec {
        default_alert_condition {
          threshold: 0.0003
        }
        feature_alert_conditions {
          key: "cnt_ad_reward"
          value {
            threshold: 0.0001
          }
        }
      }
    }
    explanation_spec {
      parameters {
        sampled_shapley_attribution {
          path_count: 2
        }
      }
      metadata {
        inputs {
          key: "cnt_ad_reward"
          value {
            input_tensor_name: "cnt_ad_reward"
            encoding: IDENTITY
            modality: "numeric"
          }
        }
        inputs {
          key: "cnt_challenge_a_friend"
          value {
            input_tensor_name: "cnt_challenge_a_friend"
            encoding: IDE

INFO:vertexai.resources.preview.ml_monitoring.model_monitors:ModelMonitoringJob projects/684754974539/locations/us-central1/modelMonitors/2411541261015056384/modelMonitoringJobs/3436327879544143872 current state:
JobState.JOB_STATE_RUNNING


### Step 5: Wait for the Model Monitoring Job to run and verify the result

#### Verify results through email

After the model monitoring job begins running, which starts after the batch prediction jobs have finished, you receive an email like the following one:

<img src="https://services.google.com/fh/files/misc/create_job_email.png" />

After the monitoring job is complete, if any anomalies are detected, you receive an email similar to the following one:

<img src="https://services.google.com/fh/files/misc/job_anomalies_email.png" />

#### Check monitoring metrics: Google Cloud Console

To view Model Monitoring metrics in the [Google Cloud Console](https://console.cloud.google.com/vertex-ai/model-monitoring/model-monitors), go to the **Monitoring** tab under **Vertex AI.**

<img src="https://storage.googleapis.com/cmm-public-data/images/bp_details.gif" />

#### Check monitoring metrics: Cloud Storage bucket

Run the following to view Model Monitoring metrics stored in the Cloud Storage bucket.  

In [24]:
try:
    my_model_monitor.show_feature_drift_stats(model_monitoring_job_1.name)
except Exception as e:
    print(e)

,Anomaly short description,Anomaly long description
Feature name,,
'cnt_completed_5_levels',High approximate Jensen-Shannon divergence between current and previous,"The approximate Jensen-Shannon divergence between current and previous is 0.549339 (up to six significant digits), above the threshold 0.3."
'cnt_level_reset_quickplay',High approximate Jensen-Shannon divergence between current and previous,"The approximate Jensen-Shannon divergence between current and previous is 0.498636 (up to six significant digits), above the threshold 0.3."
'cnt_user_engagement',High approximate Jensen-Shannon divergence between current and previous,"The approximate Jensen-Shannon divergence between current and previous is 0.0929705 (up to six significant digits), above the threshold 0.004."
'cnt_level_end_quickplay',High approximate Jensen-Shannon divergence between current and previous,"The approximate Jensen-Shannon divergence between current and previous is 0.343587 (up to six significant digits), above the threshold 0.3."
'cnt_ad_reward',High approximate Jensen-Shannon divergence between current and previous,"The approximate Jensen-Shannon divergence between current and previous is 0.672075 (up to six significant digits), above the threshold 0.3."
'cnt_use_extra_steps',High approximate Jensen-Shannon divergence between current and previous,"The approximate Jensen-Shannon divergence between current and previous is 0.390741 (up to six significant digits), above the threshold 0.3."
'cnt_post_score',High approximate Jensen-Shannon divergence between current and previous,"The approximate Jensen-Shannon divergence between current and previous is 0.352555 (up to six significant digits), above the threshold 0.3."
'cnt_level_complete_quickplay',High approximate Jensen-Shannon divergence between current and previous,"The approximate Jensen-Shannon divergence between current and previous is 0.48113 (up to six significant digits), above the threshold 0.3."
'country',High Linfty distance between current and previous,"The Linfty distance between current and previous is 0.0168421 (up to six significant digits), above the threshold 0.003. The feature value with maximum difference is: Taiwan"


In [25]:
try:
    my_model_monitor.show_feature_drift_stats(model_monitoring_job_2.name)
except Exception as e:
    print(e)

No stats were generated.


In [26]:
try:
    my_model_monitor.show_output_drift_stats(model_monitoring_job_2.name)
except Exception as e:
    print(e)

No stats were generated.


INFO:vertexai.resources.preview.ml_monitoring.model_monitors:ModelMonitoringJob projects/684754974539/locations/us-central1/modelMonitors/2411541261015056384/modelMonitoringJobs/3436327879544143872 current state:
JobState.JOB_STATE_RUNNING
INFO:vertexai.resources.preview.ml_monitoring.model_monitors:ModelMonitoringJob projects/684754974539/locations/us-central1/modelMonitors/2411541261015056384/modelMonitoringJobs/8743820040400273408 current state:
JobState.JOB_STATE_RUNNING
INFO:vertexai.resources.preview.ml_monitoring.model_monitors:ModelMonitoringJob projects/684754974539/locations/us-central1/modelMonitors/2411541261015056384/modelMonitoringJobs/3436327879544143872 current state:
JobState.JOB_STATE_RUNNING


### Step 6: Clean Up

If you no longer need your model monitoring resources, run the following to delete them:

In [27]:
# Delete the model monitor
my_model_monitor.delete(force=True)

# Delete the model
model.delete()

INFO:vertexai.resources.preview.ml_monitoring.model_monitors:Deleting ModelMonitor : projects/684754974539/locations/us-central1/modelMonitors/2411541261015056384
INFO:vertexai.resources.preview.ml_monitoring.model_monitors:Delete ModelMonitor  backing LRO: projects/684754974539/locations/us-central1/operations/6876846358678994944
INFO:vertexai.resources.preview.ml_monitoring.model_monitors:ModelMonitor deleted. . Resource name: projects/684754974539/locations/us-central1/modelMonitors/2411541261015056384
